# Running simulations through the cloud 

This notebook is a tutorial of the API used for submitting simulations to our servers.


In [1]:
import tidy3d as td
import tidy3d.web as web

# set logging level to ERROR because we'll only running simulations to demonstrate the web API, we dont care about the results.
td.config.logging_level = "ERROR"


## Setup

Let's set up a simple simulation to get started.

In [2]:
# whether to print output in web functions, note: if False (default) they will all run silently
verbose = True

# set up parameters of simulation
dl = 0.05
pml = td.PML()
sim_size = [4, 4, 4]
freq0 = 3e14
fwidth = 1e13
run_time = 1 / fwidth

# create structure
dielectric = td.Medium.from_nk(n=2, k=0, freq=freq0)
square = td.Structure(
    geometry=td.Box(center=[0, 0, 0], size=[1.5, 1.5, 1.5]), medium=dielectric
)

# create source
source = td.UniformCurrentSource(
    center=(-1.5, 0, 0),
    size=(0, 0.4, 0.4),
    source_time=td.GaussianPulse(freq0=freq0, fwidth=fwidth),
    polarization="Ex",
)

# create monitor
monitor = td.FieldMonitor(
    fields=["Ex", "Ey", "Ez"],
    center=(0, 0, 0),
    size=(td.inf, td.inf, 0),
    freqs=[freq0],
    name="field",
)

# Initialize simulation
sim = td.Simulation(
    size=sim_size,
    grid_spec=td.GridSpec.uniform(dl),
    structures=[square],
    sources=[source],
    monitors=[monitor],
    run_time=run_time,
    boundary_spec=td.BoundarySpec.all_sides(boundary=pml),
)


## Running simulation manually

For the most control, you can run the simulation through the Tidy3D web API.
Each simulation running on the server is identified by a `task_id`, which must be specified in the web API.
Let's walk through submitting a simulation this way.

In [3]:
# upload the simulation to our servers
task_id = web.upload(sim, task_name="webAPI", verbose=verbose)

# start the simulation running
web.start(task_id)

# monitor the simulation, dont move on to next line until completed.
web.monitor(task_id, verbose=verbose)

# download the results and load into a simulation data object for plotting, post processing etc.
sim_data = web.load(task_id, path="data/sim.hdf5", verbose=verbose)


11:46:37 CST Created task 'webAPI' with task_id                                 
             'fdve-d530f751-1834-40d7-b785-68cfe358cdc5' and task_type 'FDTD'.

             View task using web UI at                                          
             ]8;id=966542;https://tidy3d.simulation.cloud/workbench?taskId=fdve-d530f751-1834-40d7-b785-68cfe358cdc5\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=289820;https://tidy3d.simulation.cloud/workbench?taskId=fdve-d530f751-1834-40d7-b785-68cfe358cdc5\taskId]8;;\]8;id=966542;https://tidy3d.simulation.cloud/workbench?taskId=fdve-d530f751-1834-40d7-b785-68cfe358cdc5\=]8;;\]8;id=314572;https://tidy3d.simulation.cloud/workbench?taskId=fdve-d530f751-1834-40d7-b785-68cfe358cdc5\fdve]8;;\]8;id=966542;https://tidy3d.simulation.cloud/workbench?taskId=fdve-d530f751-1834-40d7-b785-68cfe358cdc5\-d530f751-183]8;;\
             ]8;id=966542;https://tidy3d.simulation.cloud/workbench?taskId=fdve-d530f751-1834-40d7-b785-68cfe358cdc5\4-40d7-b785-68cfe358cdc5']8;;\.

Output()

11:46:40 CST status = queued

Output()

...

Output()

11:47:02 CST status = success

             View simulation result at                                          
             ]8;id=251332;https://tidy3d.simulation.cloud/workbench?taskId=fdve-d530f751-1834-40d7-b785-68cfe358cdc5\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=202364;https://tidy3d.simulation.cloud/workbench?taskId=fdve-d530f751-1834-40d7-b785-68cfe358cdc5\taskId]8;;\]8;id=251332;https://tidy3d.simulation.cloud/workbench?taskId=fdve-d530f751-1834-40d7-b785-68cfe358cdc5\=]8;;\]8;id=879502;https://tidy3d.simulation.cloud/workbench?taskId=fdve-d530f751-1834-40d7-b785-68cfe358cdc5\fdve]8;;\]8;id=251332;https://tidy3d.simulation.cloud/workbench?taskId=fdve-d530f751-1834-40d7-b785-68cfe358cdc5\-d530f751-183]8;;\
             ]8;id=251332;https://tidy3d.simulation.cloud/workbench?taskId=fdve-d530f751-1834-40d7-b785-68cfe358cdc5\4-40d7-b785-68cfe358cdc5']8;;\.

Output()

11:47:04 CST loading simulation from data/sim.hdf5

While we broke down each of the individual steps above, one can also perform the entire process in one line by calling the [web.run()](../api/_autosummary/tidy3d.web.run.html) function as follows.

```python
sim_data = web.run(sim, task_name='webAPI', path='data/sim.hdf5')
```

(We won't run it again in this notebook to save time).

Sometimes this is more convenient, but other times it can be helpful to have the steps broken down one by one, for example if the simulation is long, you may want to [web.start](../api/_autosummary/tidy3d.web.start.html) and then exit the session and load the results at a later time using [web.load](../api/_autosummary/tidy3d.web.load.html).


## Job Container

The [web.Job](../api/_autosummary/tidy3d.web.Job.html) interface provides a more convenient way to manage single simuations, mainly because it eliminates the need for keeping track of the `task_id` and original [Simulation](../api/_autosummary/tidy3d.Simulation.html).

We can get the cost estimate of running the task before actually running it. This prevents us from accidentally running large jobs that we set up by mistake. The estimated cost is the maximum cost corresponding to running all the time steps.

In [4]:
# initializes job, puts task on server (but doesnt run it)
job = web.Job(simulation=sim, task_name="job", verbose=verbose)

# estimate the maximum cost
estimated_cost = web.estimate_cost(job.task_id)

             Created task 'job' with task_id                                    
             'fdve-6b0f356a-19d9-4537-b2d6-9b84645a2c79' and task_type 'FDTD'.

             View task using web UI at                                          
             ]8;id=508393;https://tidy3d.simulation.cloud/workbench?taskId=fdve-6b0f356a-19d9-4537-b2d6-9b84645a2c79\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=863483;https://tidy3d.simulation.cloud/workbench?taskId=fdve-6b0f356a-19d9-4537-b2d6-9b84645a2c79\taskId]8;;\]8;id=508393;https://tidy3d.simulation.cloud/workbench?taskId=fdve-6b0f356a-19d9-4537-b2d6-9b84645a2c79\=]8;;\]8;id=504999;https://tidy3d.simulation.cloud/workbench?taskId=fdve-6b0f356a-19d9-4537-b2d6-9b84645a2c79\fdve]8;;\]8;id=508393;https://tidy3d.simulation.cloud/workbench?taskId=fdve-6b0f356a-19d9-4537-b2d6-9b84645a2c79\-6b0f356a-19d]8;;\
             ]8;id=508393;https://tidy3d.simulation.cloud/workbench?taskId=fdve-6b0f356a-19d9-4537-b2d6-9b84645a2c79\9-4537-b2d6-9b84645a2c79']8;;\.

Output()

11:47:06 CST Maximum FlexCredit cost: 0.025. Minimum cost depends on task       
             execution details. Use 'web.real_cost(task_id)' to get the billed  
             FlexCredit cost after a simulation run.

While [Job](../api/_autosummary/tidy3d.web.Job.html) has methods with names identical to the web API functions above, which give some more granular control, it is often most convenient to call `.run()` so we will do that now.

In [5]:
# start job, monitor, and load results
sim_data = job.run(path="data/sim.hdf5")

11:47:07 CST status = success

Output()

11:47:09 CST loading simulation from data/sim.hdf5

Another convenient thing about [Job](../api/_autosummary/tidy3d.web.Job.html) objects is that they can be saved and loaded just like other Tidy3d components.

This is convenient if you want to save and load up the results of a job that has already finished, without needing to know the `task_id`.

In [6]:
# saves the job metadata to a single file
job.to_file("data/job.json")

# can exit session, break here, or continue in new session.

# load the job metadata from file
job_loaded = web.Job.from_file("data/job.json")

# download the data from the server and load it into a SimulationData object.
sim_data = job_loaded.load(path="data/sim.hdf5")


Output()

11:47:11 CST loading simulation from data/sim.hdf5

## Batch Processing

Commonly one needs to submit a batch of Simulations.
One way to approach this using the web API is to upload, start, monitor, and load a series of tasks one by one, but this is clumsy and you can lose your data if the session gets interrupted.

A better way is to use the built-in [Batch](../api/_autosummary/tidy3d.web.Batch.html) object.
The batch object is like a [Job](../api/_autosummary/tidy3d.web.Job.html), but stores task metadata for a series of simulations.

In [7]:
# make a dictionary of  {task name : simulation} for demonstration
sims = {f"sim_{i}": sim for i in range(3)}

# initialize a batch and run them all
batch = web.Batch(simulations=sims, verbose=verbose)

# run the batch and store all of the data in the `data/` dir.
batch_results = batch.run(path_dir="data")


11:47:12 CST Created task 'sim_0' with task_id                                  
             'fdve-fc8d45d5-4842-4698-a609-3ad4f18d34ad' and task_type 'FDTD'.

             View task using web UI at                                          
             ]8;id=950691;https://tidy3d.simulation.cloud/workbench?taskId=fdve-fc8d45d5-4842-4698-a609-3ad4f18d34ad\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=183559;https://tidy3d.simulation.cloud/workbench?taskId=fdve-fc8d45d5-4842-4698-a609-3ad4f18d34ad\taskId]8;;\]8;id=950691;https://tidy3d.simulation.cloud/workbench?taskId=fdve-fc8d45d5-4842-4698-a609-3ad4f18d34ad\=]8;;\]8;id=604440;https://tidy3d.simulation.cloud/workbench?taskId=fdve-fc8d45d5-4842-4698-a609-3ad4f18d34ad\fdve]8;;\]8;id=950691;https://tidy3d.simulation.cloud/workbench?taskId=fdve-fc8d45d5-4842-4698-a609-3ad4f18d34ad\-fc8d45d5-484]8;;\
             ]8;id=950691;https://tidy3d.simulation.cloud/workbench?taskId=fdve-fc8d45d5-4842-4698-a609-3ad4f18d34ad\2-4698-a609-3ad4f18d34ad']8;;\.

Output()

11:47:13 CST Created task 'sim_1' with task_id                                  
             'fdve-914a8883-6797-49ae-a7c3-b5cdd26a699d' and task_type 'FDTD'.

             View task using web UI at                                          
             ]8;id=722770;https://tidy3d.simulation.cloud/workbench?taskId=fdve-914a8883-6797-49ae-a7c3-b5cdd26a699d\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=166848;https://tidy3d.simulation.cloud/workbench?taskId=fdve-914a8883-6797-49ae-a7c3-b5cdd26a699d\taskId]8;;\]8;id=722770;https://tidy3d.simulation.cloud/workbench?taskId=fdve-914a8883-6797-49ae-a7c3-b5cdd26a699d\=]8;;\]8;id=150074;https://tidy3d.simulation.cloud/workbench?taskId=fdve-914a8883-6797-49ae-a7c3-b5cdd26a699d\fdve]8;;\]8;id=722770;https://tidy3d.simulation.cloud/workbench?taskId=fdve-914a8883-6797-49ae-a7c3-b5cdd26a699d\-914a8883-679]8;;\
             ]8;id=722770;https://tidy3d.simulation.cloud/workbench?taskId=fdve-914a8883-6797-49ae-a7c3-b5cdd26a699d\7-49ae-a7c3-b5cdd26a699d']8;;\.

...

11:47:18 CST Started working on Batch.

11:47:20 CST Maximum FlexCredit cost: 0.075 for the whole batch.

             Use 'Batch.real_cost()' to get the billed FlexCredit cost after the
             Batch has completed.

Output()

11:47:22 CST Batch complete.

When the batch is completed, the output is not a [SimulationData](../api/_autosummary/tidy3d.SimulationData.html) but rather a [BatchData](../api/_autosummary/tidy3d.web.BatchData.html).  The data within this [BatchData](../api/_autosummary/tidy3d.web.BatchData.html) object can either be indexed directly `batch_results[task_name]` or can be looped through `batch_results.items()` to get the [SimulationData](../api/_autosummary/tidy3d.SimulationData.html) for each task.

This was chosen to reduce the memory strain from loading all [SimulationData](../api/_autosummary/tidy3d.SimulationData.html) objects at once.

Alternatively, the batch can be looped through (several times) using the `.items()` method, similar to a dictionary.

In [8]:
# grab the sum of intensities in the simulation one by one (to save memory)
intensities = {}
for task_name, sim_data in batch_results.items():
    intensity = sim_data.get_intensity("field").sel(f=freq0)
    sum_intensity = float(intensity.sum(("x", "y")).values)
    intensities[task_name] = sum_intensity

print(intensities)


Output()

11:47:26 CST loading simulation from                                            
             data/fdve-fc8d45d5-4842-4698-a609-3ad4f18d34ad.hdf5

Output()

11:47:28 CST loading simulation from                                            
             data/fdve-914a8883-6797-49ae-a7c3-b5cdd26a699d.hdf5

Output()

11:47:30 CST loading simulation from                                            
             data/fdve-2b8d3f51-e1b3-4aaa-ac36-e27687053c8d.hdf5

{'sim_0': 6377911.0, 'sim_1': 6377911.0, 'sim_2': 6377911.0}


## Asynchronous Batching

Finally, one can make use of the [asyncio package](https://realpython.com/async-io-python/) to perform asynchronous processing of several simulations.

For this purpose, a [web.run_async](../api/_autosummary/tidy3d.web.run_async.html) function is provided, which works like the regular `web.run` but accepts a dictionary of simulations. 

Here is the previous example repeated using this feature.

In [9]:
batch_results = web.run_async(simulations=sims, verbose=verbose)


             Created task 'sim_0' with task_id                                  
             'fdve-2b428cb5-005b-4334-b489-413e267400c3' and task_type 'FDTD'.

             View task using web UI at                                          
             ]8;id=61684;https://tidy3d.simulation.cloud/workbench?taskId=fdve-2b428cb5-005b-4334-b489-413e267400c3\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=327481;https://tidy3d.simulation.cloud/workbench?taskId=fdve-2b428cb5-005b-4334-b489-413e267400c3\taskId]8;;\]8;id=61684;https://tidy3d.simulation.cloud/workbench?taskId=fdve-2b428cb5-005b-4334-b489-413e267400c3\=]8;;\]8;id=77601;https://tidy3d.simulation.cloud/workbench?taskId=fdve-2b428cb5-005b-4334-b489-413e267400c3\fdve]8;;\]8;id=61684;https://tidy3d.simulation.cloud/workbench?taskId=fdve-2b428cb5-005b-4334-b489-413e267400c3\-2b428cb5-005]8;;\
             ]8;id=61684;https://tidy3d.simulation.cloud/workbench?taskId=fdve-2b428cb5-005b-4334-b489-413e267400c3\b-4334-b489-413e267400c3']8;;\.

Output()

11:47:32 CST Created task 'sim_1' with task_id                                  
             'fdve-718578fb-ed82-4339-adf8-4703f94bada0' and task_type 'FDTD'.

             View task using web UI at                                          
             ]8;id=225403;https://tidy3d.simulation.cloud/workbench?taskId=fdve-718578fb-ed82-4339-adf8-4703f94bada0\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=308484;https://tidy3d.simulation.cloud/workbench?taskId=fdve-718578fb-ed82-4339-adf8-4703f94bada0\taskId]8;;\]8;id=225403;https://tidy3d.simulation.cloud/workbench?taskId=fdve-718578fb-ed82-4339-adf8-4703f94bada0\=]8;;\]8;id=37911;https://tidy3d.simulation.cloud/workbench?taskId=fdve-718578fb-ed82-4339-adf8-4703f94bada0\fdve]8;;\]8;id=225403;https://tidy3d.simulation.cloud/workbench?taskId=fdve-718578fb-ed82-4339-adf8-4703f94bada0\-718578fb-ed8]8;;\
             ]8;id=225403;https://tidy3d.simulation.cloud/workbench?taskId=fdve-718578fb-ed82-4339-adf8-4703f94bada0\2-4339-adf8-4703f94bada0']8;;\.

...

11:47:36 CST Started working on Batch.

11:47:38 CST Maximum FlexCredit cost: 0.075 for the whole batch.

             Use 'Batch.real_cost()' to get the billed FlexCredit cost after the
             Batch has completed.

Output()

11:47:40 CST Batch complete.

In [10]:
# grab the sum of intensities in the simulation one by one (to save memory)
intensities = {}
for task_name, sim_data in batch_results.items():
    intensity = sim_data.get_intensity("field").sel(f=freq0)
    sum_intensity = float(intensity.sum(("x", "y")).values)
    intensities[task_name] = sum_intensity

print(intensities)


Output()

11:47:43 CST loading simulation from                                            
             ./fdve-2b428cb5-005b-4334-b489-413e267400c3.hdf5

Output()

11:47:45 CST loading simulation from                                            
             ./fdve-718578fb-ed82-4339-adf8-4703f94bada0.hdf5

Output()

11:47:47 CST loading simulation from                                            
             ./fdve-82d376c5-28b6-4332-be9f-b0cba133c207.hdf5

{'sim_0': 6377911.0, 'sim_1': 6377911.0, 'sim_2': 6377911.0}


After going through this tutorial, you have learned how to run simulations with Tidy3D via the web API. If you are new to Tidy3D or the finite-difference time-domain (FDTD) method, we highly recommend going through our [FDTD101](https://www.flexcompute.com/fdtd101/) tutorials and [example library](https://www.flexcompute.com/tidy3d/examples/) before starting your own simulation adventure. 

### Asynchronous Running

We can run tasks in parallel using `Batch`, but the disadvantage is that we need to prepare our dictionary of simulations beforehand. To mitigate this, we introduced an experimental function in `tidy3d` called `td.web.async_run()` to use in place of `web.run()` but in contexts that we want to run asynchronously using `asyncio`. In short, this function will allow the interpreter to continue the execution of our function until the task has finished, allowing us to effectively batch tasks without needing to explicitly prepare them.

There is a bit of a learning curve with using `asyncio` and this [tutorial](https://realpython.com/async-io-python/) is highly recommended to understand a bit about how it works. Also, as this is an experimental feature, feel free to leave us an issue on our GitHub page if it is not working as expected on your system and we will look into it.

Here is a simple example following the same idea as the other examples above:

In [21]:
import asyncio 
td.config.logging_level = "INFO"

async def get_intensity(run_time) -> float:
    """Asynchronous function that returns the sum of intensity for a given simulation run_time."""
    sim_ = sim.updated_copy(run_time=run_time)
    sim_data = await web.async_run(sim_, task_name=f'async_rt={run_time:.2e}')
    intensity = sim_data.get_intensity("field").sel(f=freq0)
    return float(intensity.sum(("x", "y")).values)

run_times = [1e-12, 2e-12, 3e-12]

# run all intensity calculations asynchronously
intensities = await asyncio.gather(*(get_intensity(rt) for rt in run_times))


11:54:50 CST INFO: Started asynchronous task with name 'async'.

11:54:53 CST INFO: Started asynchronous task with name 'async'.

11:54:55 CST INFO: Started asynchronous task with name 'async'.

11:55:16 CST INFO: Finished asynchronous task with name 'async'.

11:55:17 CST INFO: Finished asynchronous task with name 'async'.

11:55:19 CST INFO: Finished asynchronous task with name 'async'.

In [18]:
print(intensities)

[6377911.0, 6377911.0, 6377911.0]


Let's note a few things about the above code:

1. We could pass a generic function `get_intensity` without having to prepare our set of `Simulation` objects directly.

1. `get_intensity` is asynchronous, so we need to define it with `async def`.

2. We must use the `async` version of `web.run()` called `td.web.async_run()` and `await` its return in the function.

3. We `await asyncio.gather()` passing in the coroutines (`get_intensity(rt)`) we want to process

4. All of the tasks are started before they finish, even though we didn't explicitly use `Batch`.
